## Bosques: Error OOB e importancia

### Error OOB

Usamos los datos de seguros de Caravan. Por defecto, la evaluación de OOB es sobre el porcentaje de correctos, lo que en general no es muy útil. Por ejemplo para el problema de seguros de Caravan:

In [ ]:
%autosave 0
import pandas as pd
import numpy as np
from plotnine import *
from sklearn.ensemble import RandomForestClassifier
# preprocesar
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
caravan = pd.read_csv('../datos/caravan-insurance-challenge.csv')
columnas = caravan.columns[2:86]
print(columnas)
def preprocesar_caravan(datos, tipo, columnas):
    # filtrar tipo
    datos_p = datos[datos["ORIGIN"] == tipo].copy()
    # variable respuesta
    y = datos_p["CARAVAN"].values
    datos_p = datos_p[columnas]
    datos_tipo = pd.get_dummies(datos_p.MOSHOOFD, prefix="MOSHOODFD_", drop_first = True)
    datos_p = datos_p.drop(columns = ["MOSHOOFD"])
    datos_p = pd.concat([datos_tipo, datos_p], axis = 1, sort=False)
    columnas_x = datos_p.columns
    #datos_origen = datos[datos["ORIGIN"] == tipo].drop(columns = ["ORIGIN"])
    X = datos_p.values
    return X, y, columnas_x
X_ent, y_ent, columnas_x = preprocesar_caravan(caravan, "train", columnas)
X_pr, y_pr, _ = preprocesar_caravan(caravan, "test", columnas)
print(X_ent.shape)
np.unique(y_ent, return_counts=True)

In [ ]:
rf = RandomForestClassifier(n_estimators = 1500, max_features = 60, 
                            min_samples_leaf=100, oob_score = True,
                           random_state = 10012)
ajuste = rf.fit(X_ent, y_ent)

In [ ]:
print(ajuste.oob_score_)

Aunque esta es un estimación *honesta* del error, **en general, el oob score calculado como porcentaje de correctos no es muy útil**. Solo usa corte en 0.5 y da el porcentaje de correctos.

### Importancia de variables

In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))


Por defecto, sklearn usa importancia basada en gini, no la importancia de permutaciones.

In [ ]:
importancias = ajuste.feature_importances_
pd.set_option('display.max_rows', 150)
pd.DataFrame({"variable":columnas_x, "importancia":importancias}). \
    sort_values(by = "importancia", ascending = False).round(4)

PPERSAUT: Contribution car policies
PBRAND: Contribution fire policies 
MOPLLAAG: Lower level education
APERSAUT: Number of car policies
MKOOPKLA: Purchasing power class
MOPLHOOG: High level education
MBERMIDD: Middle management
MINKGEM: Average income
MINKM30: Income < 30.000


Podemos calcular importancia de variables usando el método de permutaciones (ojo: a partir de sklearn 0.23)

In [ ]:
from sklearn.inspection import permutation_importance
# si tarda mucho, podemos tomar una muestra de los datos o reducir el número de repeticiones
r = permutation_importance(ajuste, X_ent[1:1000,:], y_ent[1:1000],
                           n_repeats=5,
                           random_state=0, scoring = "roc_auc")
importancias_df = pd.DataFrame({"variable":columnas_x, 
                                "importancia":r.importances_mean, 
                                "de":r.importances_std})

In [ ]:
importancias_df["rankeo"] = df["importancia"].rank()
importancias_df["imp_min"] = df["importancia"] - df["de"]
importancias_df["imp_max"] = df["importancia"] + df["de"]
importancias_df.sort_values(by = "importancia", ascending = False).round(4)

In [ ]:
(ggplot(importancias_df, aes("rankeo", "importancia", ymin="imp_min", ymax="imp_max"))  
  + geom_point() + geom_linerange())